# Hierarchical Differential Expression
### This notebook runs differential expression at every node of the ontology tree

# Prepare
### load necessary pacakges

In [1]:
import sys
import os 
homedir = '/data/yosef2/users/chenling/'
sys.path.append(homedir + 'scVI/')
sys.path.append(homedir + 'tabula-sapiens/')
sys.path.append(homedir + 'cellontology/')
from cellontology import CellOntolgy
import anndata
import obonet 
import pickle as pkl
from scvi.dataset import AnnDatasetFromAnnData
from utils.annotations_utils import *
from anndata import read_h5ad
import scanpy as sc

from copy import deepcopy
retrain=False

import matplotlib
%matplotlib inline

In [2]:
import networkx as nx

## set directories for reading and saving results

In [3]:
data_path = homedir + 'TabulaSapiensData/'
model_path = homedir + 'TabulaSapiens/scVImodels_final/'
res_path = homedir + 'TabulaSapiensData/combined/'
ont_path =  homedir + 'TabulaSapiensData/ontology/'


if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(model_path+'scanvi/'):
    os.mkdir(model_path+'scanvi/')
if not os.path.exists(res_path):
    os.mkdir(res_path)

In [4]:
ontology = pkl.load(open(ont_path + 'ontology.pkl', 'rb'))

In [5]:
combined = read_h5ad(data_path + 'combined/Pilot1_Pilot2.Final.h5ad')
combined.obs['propagated_cell_ontology_class'].replace('duodenum glandular cells','duodenum glandular cell',inplace=True)
combined.obs['manual_cell_ontology_class'].replace('duodenum glandular cells','duodenum glandular cell', inplace=True)

del combined.raw
combined = combined[combined.obs['propagated_cell_ontology_class']!='unassigned']

label_conversion = {}
for x in ontology.nodes:
    label_conversion[ontology.nodes[x]['name']] = x


In [56]:
celltype_id = np.asarray([label_conversion[x] for x in combined.obs['propagated_cell_ontology_class']])

# this is the function that computes the number of cells in each child node

In [57]:
def NodeInfo(ontology, root, labels):
    node_info = {}
    all_children = [x for x in nx.ancestors(ontology,root)]
    all_children = all_children 
    subset = [x in all_children for x in labels]
    if np.sum(subset)==0 or len(all_children)==0:
        return {root:(np.sum(labels==root),[])}
    count = np.sum(subset)
    groups = [x for x in ontology.predecessors(root)]
    node_info[root] = (count, groups)
    if len(groups) > 1:
        for x in groups:
            sub_node_info = NodeInfo(ontology, x, labels)
            if sub_node_info is not None:
                for y in sub_node_info.keys():
                    node_info[y] = sub_node_info[y]            
                    
    else:
        sub_node_info = NodeInfo(ontology, groups[0], labels)
        if sub_node_info is not None:
            for y in sub_node_info.keys():
                node_info[y] = sub_node_info[y]
    return node_info

In [58]:
nodeinfo = NodeInfo(ontology,'CL:0000000',celltype_id)

In [61]:
for x in nodeinfo.keys():
    print(ontology.nodes[x]['name'],x, nodeinfo[x])

cell CL:0000000 (203510, ['CL:0000003'])
native cell CL:0000003 (203510, ['CL:0000064', 'CL:0000151', 'CL:0000183', 'CL:0002371', 'CL:0011115', 'CL:0000148', 'CL:0000066', 'CL:0000584'])
ciliated cell CL:0000064 (654, ['CL:1000271'])
lung ciliated cell CL:1000271 (654, [])
secretory cell CL:0000151 (2495, ['CL:4000014', 'CL:4000024', 'CL:0008024', 'CL:1000331'])
intestinal secretory progenitor CL:4000014 (1678, [])
crypt base secretory cell CL:4000024 (66, [])
pancreatic endocrine cell CL:0008024 (547, ['CL:0000169', 'CL:0000171', 'CL:0000173'])
type B pancreatic cell CL:0000169 (52, [])
pancreatic A cell CL:0000171 (264, [])
pancreatic D cell CL:0000173 (231, [])
serous cell of epithelium of bronchus CL:1000331 (204, [])
contractile cell CL:0000183 (6637, ['CL:0000186', 'CL:0000669', 'CL:0000192'])
myofibroblast cell CL:0000186 (1166, [])
pericyte cell CL:0000669 (572, [])
smooth muscle cell CL:0000192 (1890, ['CL:0000359', 'CL:0002598'])
vascular associated smooth muscle cell CL:0000

# these are the functions that runs the DE
### WilcoxonDE run wilcoxon 
### Hierarchical DE: function to iterate over the tree and perform DE through either Wilcoxon or scVI at every node between all of its children

In [62]:
def WilcoxonDE(adata, subset, labels):
    sub_adata = adata[subset]
    sub_adata.obs['group'] = np.asarray(labels)[subset]
    if len(np.unique(sub_adata.obs['group']))==1:
        return None

    else:
        sc.pp.filter_genes(sub_adata,min_cells=10)
        sc.pp.filter_cells(sub_adata,min_counts=10)
        sc.tl.rank_genes_groups(sub_adata, groupby='group')
        return(sub_adata.uns['rank_genes_groups'])

In [63]:
def HierarchicalDE(ontology, root, labels, adata, method='Wilcoxon', posterior=None):
    print(root)
    all_children = [x for x in nx.ancestors(ontology,root)]
    all_children = all_children 
    subset = [x in all_children for x in labels]
    if len(all_children)==0:
        return None
    if np.sum(subset)==0:
        print('no cells', ontology.nodes[root]['name'])
        return None
    groups = [x for x in ontology.predecessors(root)]
    DE_res = {}
    if len(groups) > 1:
        label_group = {}
        for x in groups:
            label_group[x] = x
            groups_children = [y for y in nx.ancestors(ontology,x)]
            for y in groups_children:
                label_group[y] = x
            sub_DE_res = HierarchicalDE(ontology, x, labels, combined, method=method, posterior=posterior)
            if sub_DE_res is not None:
                for y in sub_DE_res.keys():
                    DE_res[y] = sub_DE_res[y]            
                    
        group_labels = []
        for x in labels: 
            if x in label_group.keys():
                group_labels.append(label_group[x])
            else:
                group_labels.append('None')
        
        group_labels = np.asarray(group_labels)
        
        if len(np.unique(group_labels[subset]))==1:
            print(label_group)
            print('Some groups have 0 cells resulting in only one group in comparison',ontology.nodes[root]['name'])
            sub_DE_res = HierarchicalDE(ontology, groups[0], labels, combined, method=method, posterior=posterior)
            if sub_DE_res is not None:
                for y in sub_DE_res.keys():
                    DE_res[y] = sub_DE_res[y]
            return DE_res
        else:
            print(np.sum(subset))
            if method=='Wilcoxon':
                DE_res[root] = WilcoxonDE(adata, subset, group_labels)
            elif method=='scVI':
                assert posterior is not None
                group_names, group_labels = np.unique(group_labels, return_inverse=True)
                print(np.unique(group_labels))
                a,b = posterior.one_vs_all_degenes(subset=subset, cell_labels=group_labels, mode='change')
                DE_res[root] = (a,b,group_names)
            return DE_res
    else:
        sub_DE_res = HierarchicalDE(ontology, groups[0], labels, combined, method=method, posterior=posterior)
        if sub_DE_res is not None:
            for y in sub_DE_res.keys():
                DE_res[y] = sub_DE_res[y]
        return DE_res

# Wilcoxon

In [17]:
DEres = HierarchicalDE(ontology, 'CL:0000000', combined.obs['propagated_cell_ontology_id'], combined, method='Wilcoxon')

CL:0000000
CL:0000003
CL:0000064
CL:1000271
CL:0000151
CL:4000014
CL:4000024
CL:0008024


Trying to set attribute `.obs` of view, copying.


CL:0000169
CL:0000171
CL:0000173
547


... storing 'propagated_cell_ontology_id' as categorical
... storing 'group' as categorical
Trying to set attribute `.obs` of view, copying.


CL:1000331
2495


...

... storing 'propagated_cell_ontology_id' as categorical
... storing 'group' as categorical
Trying to set attribute `.obs` of view, copying.


CL:1001428
CL:0002079
43582


... storing 'propagated_cell_ontology_id' as categorical
... storing 'group' as categorical


CL:0000584
CL:4000015
203510


Trying to set attribute `.obs` of view, copying.
... storing 'propagated_cell_ontology_id' as categorical
... storing 'group' as categorical


In [23]:
for key in DEres.keys():
    name = ontology.nodes[key]['name']
    name = name.replace(' ', '_')
    writer = pd.ExcelWriter(data_path + 'HierarchicalDE/DE.hierarchical.%s.xlsx'%name, engine='xlsxwriter')
    res = DEres[key]
    groups = res['scores'].dtype.names
    for group in groups:
        df = pd.DataFrame([res['scores'][group],
                           res['logfoldchanges'][group],res['pvals_adj'][group]], 
                           columns=res['names'][group],
                           index=['wilcoxon_score','log fold change','adjusted pvalue'])
        x = ontology.nodes[group]['name']
        if x== 'CD8-alpha-alpha-positive, alpha-beta intraepithelial T cell':
            x = 'CD8-aa+ intraepithelial T cell'
        x = x.replace('-positive','+')
        x = x.replace('-negative','-')
        x = x.replace('alpha-beta','ab')
        x = x.replace(', human','')
        x = x.replace(' cell','')
        x = x.replace('of epithelium of ','')
        x = x.replace('multipotent ', 'mp-')
        x = x.replace('multi-potent ', 'mp-')
        x = x.replace('associated', 'ass.')
        x = x.replace('epithelial of tracheobronchial tree', 'tracheobronchial epithelial')
        x = x.replace('lymphoid lineage restricted progenitor', 'lymphoid progenitor')
        df.T.to_excel(writer, sheet_name=x)
    
    writer.save()
